<p align="center">
  <img src="https://github.com/sominwadhwa/sominwadhwa.github.io/blob/master/assets/vqa/6.jpg?raw=true"/>
</p>

In [6]:
import sys
from random import shuffle, sample
import pickle as pk
import gc

import numpy as np
import pandas as pd
import scipy.io
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils
from progressbar import Bar, ETA, Percentage, ProgressBar    
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import spacy
#from spacy.en import English

from src.utils import freq_answers, grouped, get_questions_sum, get_images_matrix, get_answers_sum
from src.extract_features import get_questions_matrix_sum, get_images_matrix, get_answers_matrix

In [7]:
training_questions = open("preprocessed/v2/ques_train.txt","rb").read().decode('utf8').splitlines()
answers_train = open("preprocessed/v2/answer_train.txt","rb").read().decode('utf8').splitlines()
images_train = open("preprocessed/v2/images_coco_id.txt","rb").read().decode('utf8').splitlines()
img_ids = open('preprocessed/v2/coco_vgg_IDMap.txt').read().splitlines()
vgg_path = "data/coco/vgg_feats.mat"

In [8]:
sample(list(zip(images_train, training_questions, answers_train)), 5)

[('558851', 'Who is rowing?', 'woman'),
 ('128072', 'What shape is the plate with a fork on it?', 'round'),
 ('409199', 'Did the batter get a strike?', 'yes'),
 ('453195', 'How many people are in the photo?', '2'),
 ('316667', 'What is the word on the bottom on the round sign?', 'pizza')]

In [9]:
nlp = spacy.load("en_core_web_md")
print ("Loaded WordVec")

Loaded WordVec


In [10]:
vgg_features = scipy.io.loadmat(vgg_path)
img_features = vgg_features['feats']
id_map = dict()
print ("Loaded VGG Weights")

Loaded VGG Weights


In [11]:
lbl = LabelEncoder()
lbl.fit(answers_train)
nb_classes = len(list(lbl.classes_))
pk.dump(lbl, open('preprocessed/v2/label_encoder.sav','wb'))

In [12]:
gc.collect()

6

In [13]:
upper_lim = 1000 #Number of most frequently occurring answers in COCOVQA (Coverting >80% of the total data)
training_questions, answers_train, images_train = freq_answers(training_questions, answers_train, images_train, upper_lim)
print (len(training_questions), len(answers_train),len(images_train))

387861 387861 387861
